In [32]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import os

from keras.applications import vgg19
from keras import backend as K
import time

In [3]:
#constants

content_path = os.path.join(os.path.dirname("Neural-Style-Transfer.ipynb"), "images/Golden-Gate-Bridge.jpg")
style_path = os.path.join(os.path.dirname("Neural-Style-Transfer.ipynb"), "images/Gordon.jpg")
iterations = 10
content_weight = 0.025
style_weight = 1.0
tv_weight = 1.0

width, height = load_img(content_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [6]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [7]:
def deprocess_image(x): 
    x = x.reshape((img_nrows, img_ncols, 3))
    # shift mean of pixel to match vgg19
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # convert from BGR to RGB to match vgg19
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [9]:
content_image = K.variable(preprocess_image(content_path))
style_image = K.variable(preprocess_image(style_path))
generated_image = K.placeholder((1, img_nrows, img_ncols, 3))

In [10]:
input_tensor = K.concatenate([content_image, style_image, generated_image], axis=0)

In [ ]:
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet', include_top=False)

In [15]:
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
outputs_dict

{'input_1': <tf.Tensor 'concat:0' shape=(3, 400, 547, 3) dtype=float32>,
 'block1_conv1': <tf.Tensor 'block1_conv1/Relu:0' shape=(3, 400, 547, 64) dtype=float32>,
 'block1_conv2': <tf.Tensor 'block1_conv2/Relu:0' shape=(3, 400, 547, 64) dtype=float32>,
 'block1_pool': <tf.Tensor 'block1_pool/MaxPool:0' shape=(3, 200, 273, 64) dtype=float32>,
 'block2_conv1': <tf.Tensor 'block2_conv1/Relu:0' shape=(3, 200, 273, 128) dtype=float32>,
 'block2_conv2': <tf.Tensor 'block2_conv2/Relu:0' shape=(3, 200, 273, 128) dtype=float32>,
 'block2_pool': <tf.Tensor 'block2_pool/MaxPool:0' shape=(3, 100, 136, 128) dtype=float32>,
 'block3_conv1': <tf.Tensor 'block3_conv1/Relu:0' shape=(3, 100, 136, 256) dtype=float32>,
 'block3_conv2': <tf.Tensor 'block3_conv2/Relu:0' shape=(3, 100, 136, 256) dtype=float32>,
 'block3_conv3': <tf.Tensor 'block3_conv3/Relu:0' shape=(3, 100, 136, 256) dtype=float32>,
 'block3_conv4': <tf.Tensor 'block3_conv4/Relu:0' shape=(3, 100, 136, 256) dtype=float32>,
 'block3_pool': <t

In [16]:
def gram_matrix(x):
    assert K.ndim(x) == 3
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

In [18]:
def style_loss(style, generated):
    assert K.ndim(style) == 3
    assert K.ndim(generated) == 3
    S = gram_matrix(style)
    C = gram_matrix(generated)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [21]:
def content_loss(content, generated):
    return K.sum(K.square(generated - content))

In [20]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))

In [23]:
loss = K.variable(0.0)
content_layers = outputs_dict['block5_conv2']
content_image_features = content_layers[0, :, :, :]
generated_features = content_layers[2, :, :, :]
loss = loss + content_weight * content_loss(content_image_features, generated_features)

In [24]:
style_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in style_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    generated_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, generated_features)
    loss = loss + (style_weight / len(style_layers)) * sl
loss = loss + tv_weight * total_variation_loss(generated_image)

In [25]:
grads = K.gradients(loss, generated_image)

In [27]:
outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([generated_image], outputs)

In [28]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [29]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [30]:
evaluator = Evaluator()

In [ ]:
x = preprocess_image(content_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = generated_art + '_at_iteration_%d.png' % i
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))